<a href="https://colab.research.google.com/github/SomdeepAcharyya/Recommender-Systems/blob/main/Cross_domain_recommendation_with_user_personality.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cross Domain Recommendation with User Personality
# Hanfei Wanga, Yuan Zuo, Hong Li, Junjie Wub,

In [ ]:
# Recommendation using persoanlity of users by applying PMF on user and item latent features, 
# persoanlity obatined using 5 binary SVM classifiers on text based reviews and embedding of reviews done by PTE

In [ ]:
import pandas as pd
import scipy, cmake
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Tripadvisor review Dataset

path_of_file = r'/content/drive/MyDrive/Per_CD_RS/tripadvisor_reviews_with_country.csv'
with open(path_of_file, encoding="utf-8", errors='ignore') as infile:
  tr = pd.read_csv(infile)
tr = tr.rename(columns={"username":"userId", "taObject":"itemId"})

In [ ]:
# amazon magazine review Dataset

path = r'/content/drive/MyDrive/Per_CD_RS/Amazon_Text_Video_Games.json'
with open(path, encoding="utf-8", errors='ignore') as infile:
  az = pd.read_json(infile, lines=True, nrows=12000)
az = az.rename(columns={"reviewerID":"userId", "asin":"itemId", "overall":"rating"})

In [ ]:
# amazon review dataset magazines csv
path = r'/content/drive/MyDrive/Per_CD_RS/video_games_pers_num_300.csv'

with open(path, encoding="utf-8", errors='ignore') as infile:
  arr = pd.read_csv(infile)

In [ ]:
arr.shape
arr = arr[['0','1','2','3','4']]
arr.columns=['open', 'cons','extra','agree', 'neuro']

# Preprocessing of text of amazon dataset

In [ ]:
!pip install tweet-preprocessor

In [ ]:
import preprocessor as p
p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY,p.OPT.MENTION,p.OPT.HASHTAG, p.OPT.ESCAPE_CHAR, p.OPT.RESERVED)
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
from textblob import Word
import re
punctuation = re.compile(r'[-.?&!,:;()|0-9]')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
ad['processed_text'] = ""
ad['reviewText'].fillna(" ")
array_text = []
for i in range(len(ad)):
  x = ad['reviewText'][i]
  word_tokens = word_tokenize(x) if type(x) != float else  " "
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  lemma_words = []
  for words in filtered_sentence:
      word = Word(words).lemmatize()
      lemma_words.append(word)
  punc_words = []
  for words in lemma_words:
      word =  punctuation.sub("", words)
      if len(word) > 0:
        punc_words.append(word.lower())
  line = ""
  for i in punc_words:
    line = line + " " + i.lower()
  array_text.append(line)

In [ ]:
ad['processed_text'] = array_text

In [ ]:
ad['processed_text']

# label generation and feature processing TripAdvisor

In [ ]:
# tripadvisor dataset

In [ ]:
openness = []
for i in range(len(tr)):
  if tr['open'][i] >= tr.open.mean():
    openness.append(1)
  else:
    openness.append(0)
tr['open_label'] = openness

In [ ]:
array = []
for i in range(len(tr)):
  if tr['cons'][i] >= tr.cons.mean():
    array.append(1)
  else:
    array.append(0)
tr['cons_label'] = array

In [ ]:
array = []
for i in range(len(tr)):
  if tr['extra'][i] >= tr.extra.mean():
    array.append(1)
  else:
    array.append(0)
tr['extra_label'] = array

In [ ]:
array = []
for i in range(len(tr)):
  if tr['agree'][i] >= tr.agree.mean():
    array.append(1)
  else:
    array.append(0)
tr['agree_label'] = array

In [ ]:
array = []
for i in range(len(tr)):
  if tr['neuro'][i] >= tr.neuro.mean():
    array.append(1)
  else:
    array.append(0)
tr['neuro_label'] = array

# Predictive Text embedding

In [ ]:
from datetime import datetime
import logging
from theano import tensor as T
import theano
import string
import os
import sys
import re
import json
import random
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
!pip install libsvm
from libsvm.svmutil import *

     |████████████████████████████████| 170 kB 5.1 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp37-cp37m-linux_x86_64.whl size=233365 sha256=a7592e55fb0bac9f95dab880f14f3e566255270f55184ccf51483dbcd626b908
  Stored in directory: /root/.cache/pip/wheels/cd/e8/1e/bf95cf256e4d3ffc94289ab508c49d48e34c98220af63e3513
Successfully built libsvm


In [ ]:
X = tr['processed_text']
y = tr['neuro_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
tgt_text = ad['processed_text']
overall = tr['processed_text']
overall_l = tr['neuro_label']

In [ ]:
#tr = df.reset_index()
global json_array

In [ ]:
# Class READ

In [ ]:
class READ():
	def __init__(self,window_size):
		self.w={}  	# word and their vertes id list
		self.wcount={} #list
		self.listoffreq=[]
		self.d=[]	# document and their vertex id
		self.l={}	# class label and their vertex id list
		self.w2w={}	#word to word mapping with edge weight list
		self.w2d={}	#word to document mapping with edge weight list
		self.w2l={}	#word to class mapping with edge weight list
		self.ndocs=0	#no of document 
		self.nlabel=0	#no of labels
		self.window_size=window_size;
		self.nedge=0
		self.nvertex=0

	def generate_graphs(self):
		files = []
		labels = []
		for filename in range(len(overall)):
			files.append(overall[filename])
			if overall_l[filename] == 1:
				labels.append('pos')
			elif overall_l[filename] == 0:
				labels.append('neg')
        
		document=1
		nodes=1
		label=1
		for x in labels:
			if x not in self.l:
				self.l[x]=label
				label+=1
		self.nlabel = label

		for filename in files:
			fd=filename if type(filename)!= float else " "
			for x in fd.split():
				x = x.strip()
				x=x.split()
				#if x[-1][-1:]=='\n':
				#	x[-1]=x[-1][:-1]
				self.d.append((x,labels[files.index(filename)],document))
				document+=1
				for y in x:
					if y not in self.w:
						self.w[y]=nodes
						self.wcount[nodes]=1
						nodes+=1
					else:
						self.wcount[self.w[y]]+=1
		self.ndocs = document
		self.nvertex = nodes-1

		i=1
		# print self.nvertex
		while i <=self.nvertex:
			self.listoffreq.append(self.wcount[i])
			i= i+1
		
		for x in self.d:
			index=0
			len1=len(x[0])
			label=self.l[x[1]]
			for y in x[0]:
				word=self.w[y] #word is vertex id of "word and document" graph
				left=index-self.window_size/2
				right=index+self.window_size/2
				if left<0:
					left=0
				if right>len1:
					right=len1-1

				# word to word dictionary
				for z in range(left,index):
					a=self.w[x[0][z]]
					# print a,word
					if a==word:
						continue
					if word not in self.w2w:
						self.w2w[word]={}
					b=self.w2w[word]
					if a not in b:
						b[a]=1
					else:
						b[a]=b[a]+1
				
				for z in range(index+1,right):
					# print x[0][z],y,z
					a=self.w[x[0][z]]
					if a==word:
						continue
					if word not in self.w2w:
						self.w2w[word]={}
					b=self.w2w[word]
					if a not in b:
						b[a]=1
					else:
						b[a]=b[a]+1

				# document to word  dictionary
				if x[2] not in self.w2d:
					self.w2d[x[2]]={}
				if word not in self.w2d[x[2]]:
					self.w2d[x[2]][word]=0
				self.w2d[x[2]][word]=self.w2d[x[2]][word]+1				
				
				# word to label dictionary
				if label not in self.w2l:
					self.w2l[label]={}
				if word not in self.w2l[label]:
					self.w2l[label][word]=0
				self.w2l[label][word] = self.w2l[label][word]+1
				index+=1

		self.json_array = self.w #json.dump(np.array(self.w), open('word_mapping.json', 'wb'))
  # json.dump(self.l, open('label_mapping.json', 'wb'))
  #       json.dump(self.d, open('document_mapping.json', 'wb'))
  #       print 'w2l', len(self.w2l.keys())
  #       print 'w2d', len(self.w2d.keys())
  #       print 'w2w', len(self.w2w.keys())

	def gen_edgeprob(self):
		p = []	#probability of edge b/w w1-w2
		v1 = []	#w1
		v2 = [] #w2
		pd = [] #probability of edge b/w w-d
		v3 = [] #d
		v4 = []	#w
		pl = [] #probability of edge b/w w-l
		v5 = [] #l
		v6 = [] #w
		for k in self.w2w.keys():
			for kj in self.w2w[k].keys():
				p.append(self.w2w[k][kj])#all co-occurences added
				v1.append(k)
				v2.append(kj)
				self.nedge += 1
		p = np.asarray(p, dtype=np.float64)
		nP=np.asarray(self.listoffreq, dtype=np.float64)
		nP = np.power(nP, (3.0/4.0))
		p = p / float(sum(p))
		nP = nP / float(sum(nP))

		for doc in self.w2d.keys():
			for word in self.w2d[doc].keys():
				pd.append(self.w2d[doc][word])#frequency of that word in the doc
				v3.append(doc)
				v4.append(word)
		pd = np.asarray(pd, dtype=np.float64)
		pd = pd / float(sum(pd))

		for label in self.w2l.keys():
			for word in self.w2l[label].keys():
				pl.append(self.w2l[label][word])#frequency of that word in that label
				v5.append(label)
				v6.append(word)
		pl = np.asarray(pl, dtype=np.float64)
		pl = pl / float(sum(pl))
		
		return p, nP, v1, v2, pd, v3, v4, pl, v5, v6

In [ ]:
# class PTE

In [ ]:
from theano import tensor as T
import theano
import numpy as np

class PTE(object):
    '''
    Defines the PTE model (cost function, parameters in theano.
    '''
    def __init__(self, nvertex, out_dim, ndocs, nlabels, lr=0.05):
        '''
        Parameters specs:
            nvertex : no of vertices in the graph
            out_dim : node vector dimension
            ndocs : number of documents in the corpus
            nlabels : number of labels
            lr : learning rate.
        '''
        # TO-DO: Try initialization from uniform
        # Initialize model paramters
        eps = np.sqrt(1.0 / float(out_dim))
        self.w = np.asarray(np.random.uniform(low=-eps, high=eps, size=(nvertex, out_dim)),
                       dtype=theano.config.floatX)
        self.d = np.asarray(np.random.uniform(low=-eps, high=eps, size=(ndocs, out_dim)),
                       dtype=theano.config.floatX)
        self.l = np.asarray(np.random.uniform(low=-eps, high=eps, size=(nlabels, out_dim)),
                       dtype=theano.config.floatX)
        self.W = theano.shared(self.w, name='W', borrow=True)
        self.D = theano.shared(self.d, name='D', borrow=True)
        self.L = theano.shared(self.l, name='L', borrow=True)
        self.lr = lr

    def ww_model(self):
        '''
        Performs SGD update (pre-training on ww graph).
        '''
        indm = T.iscalar()
        indc = T.iscalar()
        indr = T.ivector()		#vector of 5 negative edge samples
        Uj = self.W[indm, :] #one row of W
        Ui = self.W[indc, :] #one row of W
        Ui_Set = self.W[indr, :] #rows for negative edge sampling
        cost_ww = T.log(T.nnet.sigmoid(T.dot(Ui, Uj)))
        cost_ww -= T.log(T.sum(T.nnet.sigmoid(T.sum(Uj * Ui_Set, axis=1))))
        #cost_ww = T.dot(Ui, Uj)
        #cost_ww -= T.log(T.sum(T.exp(T.sum(Uj * Ui_Set, axis=1))))
        cost = -cost_ww
        grad_ww = T.grad(cost, [Uj, Ui, Ui_Set]) #gradient w.r.t 3 variables
        deltaW = T.inc_subtensor(self.W[indm, :], - (self.lr) * grad_ww[0])
        deltaW = T.inc_subtensor(deltaW[indc, :], - (self.lr) * grad_ww[1])
        deltaW = T.inc_subtensor(deltaW[indr, :], - (self.lr) * grad_ww[2])
        updateD = [(self.W, deltaW)]
        self.train_ww = theano.function(inputs=[indm, indc, indr], outputs=cost, updates=updateD)

    def pretraining_ww(self, indm, indc, indr):
        return self.train_ww(indm, indc, indr)

    def wd_model(self):
        '''
        Performs SGD update (pre-training on wd graph).
        '''
        indm = T.iscalar()
        indc = T.iscalar()
        indr = T.ivector()		#vector of 5 negative edge samples
        Uj = self.D[indm, :] #one row of D
        Ui = self.W[indc, :] #one row of W
        Ui_Set = self.W[indr, :] #rows of W for negative edge sampling
        cost_wd = T.log(T.nnet.sigmoid(T.dot(Ui, Uj)))
        cost_wd -= T.log(T.sum(T.nnet.sigmoid(T.sum(Uj * Ui_Set, axis=1))))
        cost = -cost_wd
        grad_wd = T.grad(cost, [Uj, Ui, Ui_Set]) #gradient w.r.t 3 variables
        
        deltaD = T.inc_subtensor(self.D[indm, :], - (self.lr) * grad_wd[0])
        deltaW = T.inc_subtensor(self.W[indc, :], - (self.lr) * grad_wd[1])
        deltaW = T.inc_subtensor(deltaW[indr, :], - (self.lr) * grad_wd[2])
        updateD = [(self.W, deltaW), (self.D, deltaD)]
        self.train_wd = theano.function(inputs=[indm, indc, indr], outputs=cost, updates=updateD)

    def pretraining_wd(self, indm, indc, indr):
        return self.train_wd(indm, indc, indr)

    def wl_model(self):
        '''
        Performs SGD update (pre-training on wd graph).
        '''
        indm = T.iscalar()
        indc = T.iscalar()
        indr = T.ivector()		#vector of 5 negative edge samples
        Uj = self.L[indm, :] #one row of L
        Ui = self.W[indc, :] #one row of W
        Ui_Set = self.W[indr, :] #rows of W for negative edge sampling
        cost_wl = T.log(T.nnet.sigmoid(T.dot(Ui, Uj)))
        cost_wl -= T.log(T.sum(T.nnet.sigmoid(T.sum(Uj * Ui_Set, axis=1))))
        cost = -cost_wl
        grad_wl = T.grad(cost, [Uj, Ui, Ui_Set]) #gradient w.r.t 3 variables
        
        deltaL = T.inc_subtensor(self.L[indm, :], - (self.lr) * grad_wl[0])
        deltaW = T.inc_subtensor(self.W[indc, :], - (self.lr) * grad_wl[1])
        deltaW = T.inc_subtensor(deltaW[indr, :], - (self.lr) * grad_wl[2])
        updateD = [(self.W, deltaW), (self.L, deltaL)]
        self.train_wl = theano.function(inputs=[indm, indc, indr], outputs=cost, updates=updateD)

    def pretraining_wl(self, indm, indc, indr):
        return self.train_wl(indm, indc, indr)

    def save_model(self):
        '''
        Save embedding matrices on disk
        '''
        W = self.W.get_value()
        np.save('lookupW', W)

In [ ]:
# class train

In [ ]:
class train_pte(object):

	#Initialize graph
	def __init__(self):
		self.window_size=300
		self.graphs = READ(self.window_size) #Give window size as parameter
		self.graphs.generate_graphs()
		print("Read and graphing complete")
		self.ndims = 300
		self.lr = 0.005
		self.batch_size = 100
		self.k = 6
		self.nepochs = 1

	def train(self):
		p, nP, v1, v2, pd, v3, v4, pl, v5, v6  = self.graphs.gen_edgeprob()
		pte = PTE(self.graphs.nvertex, self.ndims, self.graphs.ndocs, self.graphs.nlabel)
		pte.ww_model()
		pte.wd_model()
		pte.wl_model()
		currentHour = datetime.utcnow().hour

		# setting up logger
		logger = logging.getLogger("wordTovec")
		logger.setLevel(logging.INFO)
		logger.setLevel(logging.DEBUG)
		fh = logging.FileHandler("word2graph2vec.log")
		formatter = logging.Formatter('%(asctime)s %(message)s')
		fh.setFormatter(formatter)
		logger.addHandler(fh)
		logger.info("Training started")
		logger.info("Total edges : %f " % self.graphs.nedge)

		for it in range(0, self.graphs.nedge, self.batch_size):
			sample = np.random.choice(p.shape[0], self.batch_size, p=p)
			k=0
			while k < sample.shape[0]:
				i = v1[sample[k]]-1
				j = v2[sample[k]]-1
				i_set = np.asarray(np.random.choice(self.graphs.nvertex, size=self.k, p=nP), dtype=np.int32)
				if i in i_set:
					i_set = np.delete(i_set, np.where(i_set==i))
				costww = pte.pretraining_ww(j, i, i_set)
				k = k + 1
			
			sample = np.random.choice(pd.shape[0], self.batch_size, p=pd)
			k=0
			while k < sample.shape[0]:
				i = v4[sample[k]]-1
				j = v3[sample[k]]-1
				i_set = np.asarray(np.random.choice(self.graphs.nvertex, size=self.k, p=nP), dtype=np.int32)
				if i in i_set:
					i_set = np.delete(i_set, np.where(i_set==i))
				costwd = pte.pretraining_wd(j, i, i_set)
				k = k+1

			sample = np.random.choice(pl.shape[0], self.batch_size, p=pl)
			k=0
			while k < sample.shape[0]:
				i = v6[sample[k]]-1	#one word
				j = v5[sample[k]]-1	#one label
				i_set = np.asarray(np.random.choice(self.graphs.nvertex, size=self.k, p=nP), dtype=np.int32)
				if i in i_set:
					i_set = np.delete(i_set, np.where(i_set==i))
				costwl = pte.pretraining_wl(j, i, i_set)
				k = k+1

			#print("Current it: ", it, " complete of total: ", self.graphs.nedge)
			if datetime.utcnow().hour >= currentHour+2:
				logger.info("ww Cost after 2 hrs training is %f" % costww)
				logger.info("wd Cost after 2 hrs training is %f" % costwd)
				logger.info("wl Cost after 2 hrs training is %f" % costwl)
				logger.info("Current it: %f " % it)
				logger.info("Saving the model")
				pte.save_model()
				currentHour += 2
		logger.info("Saving the model finally")
		print("saving the model finally")
		pte.save_model()


if __name__ == "__main__":
	pte = train_pte()
	pte.train()

Read and graphing complete
saving the model finally


In [ ]:
# test class

In [ ]:
class test_pte(object):

  def __init__(self, train_text, train_labels, test_text, test_labels, tgt_text):
    self.all_words = pte.graphs.w
    self.train_documents = []
    self.train_text = train_text
    self.train_labels = train_labels
    self.test_text = test_text
    self.test_documents = []
    self.test_labels = test_labels
    self.test_predicted_labels = []
    self.tgt_text = tgt_text
    self.tgt_predicted_labels = []
    self.tgt_documents = []
    self.tr_f1 = 0.0
    self.tr_acc = 0.0
    self.ts_f1 = 0.0
    self.ts_acc = 0.0

    self.word_embedding = np.load('lookupW.npy')
    self.D = 300

  def load_train_data(self):
    train_set = []
    document_no = 1
    files = np.array(self.train_text)
    class_labels = ['pos','neg']
    index = 0
    for file_name in files:
      #fp = open(filename)
      lines = file_name
      words = lines.split(" ") if type(lines) != float else " "
      document = (words,class_labels[index],document_no)
      train_set.append(document)
      document_no += 1
    index+=1
    return train_set

  def load_test_data(self):
    test_set = []
    document_no = 1
    files = np.array(self.test_text)
    class_labels = ['pos','neg']
    index = 0
    for file_name in files:
      #fp = open(filename)
      lines = str(file_name)
      words = lines.split(" ") if type(lines) != float else " "
      document = (words,class_labels[index],document_no)
      test_set.append(document)
      document_no += 1
    index+=1
    return test_set

  def load_tgt_data(self):
    tgt_set = []
    document_no = 1
    files = np.array(self.tgt_text)
    class_labels = ['pos','neg']
    index = 0
    for file_name in files:
      #fp = open(filename)
      lines = str(file_name)
      words = lines.split(" ") if type(lines) != float else " "
      document = (words,class_labels[index],document_no)
      tgt_set.append(document)
      document_no += 1
    index+=1
    return tgt_set
  
  def test(self):
    train_set = self.load_train_data()
    test_set = self.load_test_data()
    tgt_set = self.load_tgt_data()

    for index in range(len(train_set)):
      document_sum = np.zeros(self.D)
      for word_index in range(len(train_set[index][0])-1):
        word = train_set[index][0][word_index+1]
        i = self.all_words[word]
        embedding = self.word_embedding[i-1]
        document_sum = np.add(document_sum, embedding)
      document_average = np.divide(document_sum, len(train_set[index][0]))
      self.train_documents.append(document_average)
      #self.train_labels.append(train_set[index][1])
      self.train_labels = np.array(self.train_labels)

    for index in range(len(test_set)):
      document_sum = np.zeros(self.D)
      for word_index in range(len(test_set[index][0])-1):
          word = test_set[index][0][word_index+1]
          try:
            i = self.all_words[word]
          except:
            i = 0
          embedding = self.word_embedding[i-1]
          document_sum = np.add(document_sum, embedding)
      document_average = np.divide(document_sum, len(test_set[index][0]))
      self.test_documents.append(document_average)
      #self.test_labels.append(test_set[index][1])
      self.test_actual_labels = np.array(self.test_labels)    

    for index in range(len(tgt_set)):
      document_sum = np.zeros(self.D)
      for word_index in range(len(tgt_set[index][0])-1):
          word = tgt_set[index][0][word_index+1]
          try:
            i = self.all_words[word]
          except:
            i = 0
          embedding = self.word_embedding[i-1]
          document_sum = np.add(document_sum, embedding)
      document_average = np.divide(document_sum, len(tgt_set[index][0]))
      self.tgt_documents.append(document_average)

    clf = svm.SVC()
    clf.fit(self.train_documents, self.train_labels)
    
    
    from sklearn.metrics import f1_score, accuracy_score
    self.train_predicted_labels = clf.predict(self.train_documents)
    self.tr_f1 = f1_score(self.train_labels, self.train_predicted_labels)
    print("f1 score_train: ", self.tr_f1)
    self.tr_acc = accuracy_score(self.train_labels, self.train_predicted_labels)
    print("acc score_train: ", self.tr_acc)
    self.test_predicted_labels = clf.predict(self.test_documents)
    self.ts_f1 = f1_score(self.test_actual_labels, self.test_predicted_labels)
    print("f1 score_test: ", self.ts_f1)
    self.ts_acc = accuracy_score(self.test_actual_labels, self.test_predicted_labels)
    print("acc score_test: ", self.ts_acc)

    self.tgt_predicted_labels = clf.predict(self.tgt_documents)

if __name__ == "__main__":
  pte_test = test_pte(X_train, y_train, X_test, y_test, tgt_text)
  pte_test.test()

f1 score_train:  0.8086867709509218
acc score_train:  0.8369525380245556
f1 score_test:  0.5128979781547757
acc score_test:  0.6101190476190477


In [ ]:
tgt_pers_neuro = pte_test.tgt_predicted_labels
tgt_pers_neuro

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
list(tgt_pers_open).count(1)

11002

In [ ]:
column_names = ["open", "cons", "extra", 'agree', 'neuro']
df = pd.DataFrame(columns = column_names)
df.open = tgt_pers_open
df.cons = tgt_pers_cons
df.extra = tgt_pers_extra
df.agree = tgt_pers_agree
df.neuro = tgt_pers_neuro

In [ ]:
df.to_csv(r'/content/drive/MyDrive/Per_CD_RS/amazon_movies_pers_pte_binary.csv', encoding = 'utf-8-sig') 

# Probabilistic matrix factorization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy.stats import pearsonr

In [ ]:
#arr = arr[0:12000]
df = pd.DataFrame(arr)

In [ ]:
df['userId'] = az['userId']
df['itemId'] = az['itemId']
df['rating'] = az['rating']

In [ ]:
df.columns = ['open', 'cons', 'extra', 'agree', 'neuro', 'userId', 'itemId', 'rating'] 

In [ ]:
df

In [ ]:
    user_to_row = {}
    item_to_column = {}
    n_dims = 10
    n_features = 5
    parameters = {}
    train_size = 0.8

    uniq_users = np.unique(df.userId)
    uniq_items = np.unique(df.itemId)

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j
    
    n_users = len(uniq_users)
    n_items = len(uniq_items)
    
    R = np.zeros((n_users, n_items))
    
    df_copy = df.copy()
    train_set = df_copy.sample(frac=train_size).reset_index()
    user_features_train = np.array(train_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
    test_set = df_copy.drop(train_set.index).reset_index()
    user_features_test = np.array(test_set[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
    user_features_unique = np.array(df.groupby(by='userId').mean()[['open', 'cons', 'extra', 'agree', 'neuro']].fillna(0))
    
    for index, row in train_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        R[i, j] = row.rating
    print(R.shape)

(10633, 170)


In [ ]:
rating =[]
for i in range(n_users):
  rating.append(np.average(R[i]))

In [ ]:
# DO NOT RUN THIS

def get_ratings_matrix(df, train_size=0.75):
    user_to_row = {}
    item_to_column = {}
    df_values = df.values
    n_dims = 10
    n_features = 5
    parameters = {}
    
    uniq_users = np.unique(df_values[:, 3])
    uniq_items = np.unique(df_values[:, 0])

    for i, user_id in enumerate(uniq_users):
        user_to_row[user_id] = i

    for j, item_id in enumerate(uniq_items):
        item_to_column[item_id] = j
    
    n_users = len(uniq_users)
    n_items = len(uniq_items)
    
    R = np.zeros((n_users, n_items))
    
    df_copy = df.copy()
    train_set = df_copy.sample(frac=train_size, random_state=0)
    test_set = df_copy.drop(train_set.index)
    
    for index, row in train_set.iterrows():
        i = user_to_row[row.userId]
        j = item_to_column[row.itemId]
        R[i, j] = row.rating

    return R, train_set, test_set, n_dims, n_users, n_items, user_to_row, item_to_column, n_features

R, train_set, test_set, n_dims, n_users, n_items, user_to_row, item_to_column, n_features = get_ratings_matrix(tr, 0.8)
parameters = {}

In [ ]:
def initialize_parameters(lambda_U, lambda_V, lambda_W, user_features):
    U = np.random.normal(0.0, 1.0 / lambda_V, (n_dims, n_users))    #np.zeros((n_dims, n_users), dtype=np.float64)
    V = np.random.normal(0.0, 1.0 / lambda_V, (n_dims, n_items))
    print(len(rating), len(user_features))
    op, _ = pearsonr(rating, user_features[:,0])
    cn, _ = pearsonr(rating, user_features[:,1])
    ex, _ = pearsonr(rating, user_features[:,2])
    ag, _ = pearsonr(rating, user_features[:,3])
    ne, _ = pearsonr(rating, user_features[:,4])
    W = [op, cn, ex, ag, ne]     # for persoanlity boosted PMF 5 persoanlity traits n_features=5
    
    
    parameters['U'] = U
    parameters['V'] = V
    parameters['W'] = W   ##
    parameters['lambda_U'] = lambda_U
    parameters['lambda_V'] = lambda_V
    parameters['lambda_W'] = lambda_W  ##

In [ ]:
def update_parameters():
    U = parameters['U']
    V = parameters['V']
    W = parameters['W']  ##
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']
    lambda_W = parameters['lambda_W']   ##

    for i in range(n_users):
        V_j = V[:, R[i, :] > 0]
        U[:, i] = np.dot(np.linalg.inv(np.dot(V_j, V_j.T) + lambda_U * np.identity(n_dims)), np.dot(R[i, R[i, :] > 0], V_j.T))
      
    for j in range(n_items):
        U_i = U[:, R[:, j] > 0]
        V[:, j] = np.dot(np.linalg.inv(np.dot(U_i, U_i.T) + lambda_V * np.identity(n_dims)), np.dot(R[R[:, j] > 0, j], U_i.T))
           
    parameters['U'] = U
    parameters['V'] = V
    parameters['W'] = W ##

In [ ]:
def update_max_min_ratings(user_features):
    U = parameters['U'].T
    V = parameters['V']
    W = parameters['W']  ##
    P = user_features  ##

##
    for i in range(n_users):
      t = 0
      for k in range(n_features):
        t += P[i][k] * W[k]
      t = t / np.sum(P[i][k])
      U[i] = np.add(U[i], t)

    R = U @ V
    R[R>5] = 5
    R[R<0] = 0
    R = np.nan_to_num(R)
    min_rating = np.min(R)
    max_rating = np.max(R)

    parameters['min_rating'] = min_rating
    parameters['max_rating'] = max_rating

In [ ]:
def log_a_posteriori(user_features):
    lambda_U = parameters['lambda_U']
    lambda_V = parameters['lambda_V']
    lambda_W = parameters['lambda_W'] ##
    U = parameters['U']
    V = parameters['V']
    W = parameters['W']  ##
    P = user_features    ##

    UV = np.dot(U.T, V)
    R_UV = (R[R > 0] - UV[R > 0])
    W = np.array(W)
    
    return -0.5 * (np.sum(np.dot(R_UV, R_UV.T)) + lambda_U * np.sum(np.dot(U, U.T)) + lambda_V * np.sum(np.dot(V, V.T)) + lambda_W * np.sum(np.dot(W, W.T)))  ##


In [ ]:
def predict(user_id, item_id, user_features):
    U = parameters['U']
    V = parameters['V']
    W = parameters['W']   ##
    P = user_features     ##
  
    r_ij = U[:, user_to_row[user_id]].reshape(1, -1) @ V[:, item_to_column[item_id]].reshape(-1, 1)
    r_ij[r_ij<0] = 0
    r_ij[r_ij>5] = 5
    r_ij = np.nan_to_num(r_ij)

    max_rating = parameters['max_rating']
    min_rating = parameters['min_rating']

    return 0 if max_rating == min_rating else ((r_ij[0][0] - min_rating) / (max_rating - min_rating)) * 5.0

In [ ]:
def evaluate(dataset, user_features):
    ground_truths = []
    predictions = []
    
    for index, row in dataset.iterrows():
        ground_truths.append(row.loc['rating'])
        t = predict(row.loc['userId'], row.loc['itemId'], user_features)
        predictions.append(t)   ##
    
    print("mae", mean_absolute_error(ground_truths, predictions), "rmse", mean_squared_error(ground_truths, predictions))
    return mean_squared_error(ground_truths, predictions, squared=False)

In [ ]:
def train(n_epochs, user_features_train, user_features_test):
    user_features_total = np.array(pd.concat([pd.DataFrame(user_features_train), pd.DataFrame(user_features_test)], axis=0))  ##
    initialize_parameters(0.02, 0.02, 0.01, user_features_unique)   ##
    print("initialization done")
    log_aps = []
    rmse_train = []
    rmse_test = []


    update_max_min_ratings(user_features_total)
    print("update_max_min_ratings 1 done")
    rmse_train.append(evaluate(train_set, user_features_train))
    rmse_test.append(evaluate(test_set, user_features_test))
    print("rmse done")
    #print("train", rmse_train)
    #print("test", rmse_test)
    print("starting epochs")
    
    for k in range(n_epochs):
        update_parameters()
        log_ap = log_a_posteriori(user_features_total)  ##
        log_aps.append(log_ap)

        if (k + 1) % 10 == 0:
            rmse_train.append(evaluate(train_set, user_features_train))    ##
            rmse_test.append(evaluate(test_set, user_features_test))       ##
            print('Log p a-posteriori at iteration', k + 1, ':', log_ap)

    update_max_min_ratings(user_features_total)

    return log_aps, rmse_train, rmse_test

In [ ]:
log_ps, rmse_train, rmse_test = train(150, user_features_train, user_features_test)   ##

10633 10633
initialization done
update_max_min_ratings 1 done
mae 2.5261159098508505 rmse 11.136698817888746
mae 2.568466626197096 rmse 11.415842397952646
rmse done
starting epochs
mae 0.13999034860506201 rmse 0.025139143415427705
mae 0.8807769989948099 rmse 3.6151958616617255
Log p a-posteriori at iteration 10 : -3733.0922529215286
mae 0.08114585122179235 rmse 0.009179188546956847
mae 0.8362474456841809 rmse 3.554941842226393
Log p a-posteriori at iteration 20 : -2214.5580968426934
mae 0.05775608875336177 rmse 0.005388289054980004
mae 0.8160951324928959 rmse 3.5326428107867134
Log p a-posteriori at iteration 30 : -1581.6044894651936
mae 0.04500857803085351 rmse 0.0038965611625478153
mae 0.8055154739688309 rmse 3.5226010407753496
Log p a-posteriori at iteration 40 : -1242.5500201655311
mae 0.03695890701344735 rmse 0.0031569561292444986
mae 0.797642911611064 rmse 3.509422927926637
Log p a-posteriori at iteration 50 : -1033.4722918052962
mae 0.03140377193975204 rmse 0.002737237766451939


In [ ]:
_, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
plt.title('Training results')
ax1.plot(np.arange(len(log_ps)), log_ps, label='MAP')
ax1.legend()

ax2.plot(np.arange(len(rmse_train)), rmse_train, label='RMSE train')
ax2.plot(np.arange(len(rmse_test)), rmse_test, label='RMSE test')
ax2.legend()

plt.show()

In [ ]:
print('RMSE of training set:', evaluate(train_set, user_features_train))
print('RMSE of testing set:', evaluate(test_set, user_features_test))

mae 0.19970311350209957 rmse 0.1289247175134238
RMSE of training set: 0.3590608827391586
mae 0.9291509301265921 rmse 2.5314017249292347
RMSE of testing set: 1.5910379395002605


In [ ]:
# amazon_movies
RMSE of training set: 4.547824571521348
RMSE of testing set: 4.428976555970164

In [ ]:
rmse_test

[4.717906271512138,
 2.1908230464091267,
 2.1838680034694447,
 2.18200862070802,
 2.1811151288994233,
 2.1804271788358207,
 2.179750123552533,
 2.1790497853093656,
 2.17841697917851,
 2.177901897141428,
 2.1774771992475954,
 2.177123424493576,
 2.1768278356652093,
 2.1765807730094684,
 2.1763739843915357,
 2.1761954772126346]

In [ ]:
rmse_train

[4.640758457769598,
 0.19445396005030074,
 0.12115428247808824,
 0.09819568547917262,
 0.08795667070273773,
 0.08254120601665235,
 0.0793591721139332,
 0.07734787590764923,
 0.07600624221639128,
 0.07507360671790234,
 0.07440378572402324,
 0.07390977547111628,
 0.07353729198368637,
 0.07325114500097318,
 0.07302777547305278,
 0.07285097264121854]

In [ ]:
# with personality
RMSE of training set: 0.8434822554410405
RMSE of testing set: 1.0651675581716595

In [ ]:
tr[['userId', 'open', 'cons', 'extra', 'agree', 'neuro']].groupby(by='userId').mean()

In [ ]:
print(max(tr['open']), min(tr['open']), max(tr['open'])-min(tr['open']))

0.67193 0.42623 0.24570000000000003
